# Tensor Hypercontraction

SELECT and PREPARE for the molecular tensor hypercontraction (THC) hamiltonian

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq
from typing import *
import numpy as np

## `UniformSuperpositionTHC`
Prepare uniform superposition state for THC.

$$
    |0\rangle^{\otimes 2\log(M+1)} \rightarrow \sum_{\mu\le\nu}^{M} |\mu\rangle|\nu\rangle + \sum_{\mu}^{N/2}|\mu\rangle|\nu=M+1\rangle,
$$

where $M$ is the THC auxiliary dimension, and $N$ is the number of spin orbitals.

The toffoli complexity of this gate should be 10 * log(M+1) + 2 b_r - 9.
Currently it is a good deal larger due to:
    1. inverting inequality tests should not need more toffolis.
    2. We are not using phase-gradient gate toffoli cost for Ry rotations

#### Parameters
 - `num_mu`: THC auxiliary index dimension $M$
 - `num_spin_orb`: number of spin orbitals $N$ 

Registers:
- mu: $\mu$ register.
- nu: $\nu$ register.
- succ: ancilla flagging success of amplitude amplification.
- eq_nu_mp1: ancillas for flagging if $\nu = M+1$.

#### References
[Even more efficient quantum computations of chemistry through tensor hypercontraction](https://arxiv.org/pdf/2011.03494.pdf). Eq. 29.


In [ ]:
from qualtran.bloqs.chemistry.thc import UniformSuperpositionTHC

num_mu = 10
num_spin_orb = 4
uniform_bloq = UniformSuperpositionTHC(num_mu=num_mu, num_spin_orb=num_spin_orb)
show_bloq(uniform_bloq)

In [ ]:
from qualtran.resource_counting import get_bloq_counts_graph, GraphvizCounts
from qualtran.bloqs.chemistry.thc_notebook_utils import generalize
graph, sigma = get_bloq_counts_graph(uniform_bloq, generalizer=generalize)
GraphvizCounts(graph).get_svg()

Let's print out the costs contributing to the TCount.

In [ ]:
from qualtran.bloqs.chemistry.thc_notebook_utils import bin_bloq_counts

binned_counts = bin_bloq_counts(uniform_bloq)
# number of bits for mu register (nm in THC paper)
nm = uniform_bloq.signature[0].bitsize
# Costs for THC paper
# The factor of 4 is for Toffoli -> T conversion
paper_costs = {
    'comparator': sum([4*4*(nm - 1), 4*(4*nm - 3)]), # 4 comparitors of cost nm - 1 Toffolis
    'rotation': sum([4*4, 4*4]), # Given as br - 3, br = 7 is the number of bits of precision for rotations.
    'reflections': sum([4*3, 4*(2*nm-1)]), # 5 qubit reflection for comparitors and 2*nm + 1 qubits reflect after hadamards
    'other': sum([4*3]), # "Checking the inequality test" unclear if this is the multi-control not gate.
}
for k, v in paper_costs.items():
    print(f"{k}: qualtran = {binned_counts[k]} vs paper cost = {v}.")

print(sum(v for v in binned_counts.values()))

The discrepancies arise from the following issues:

1. Comparators: The paper uncomputes the comparators at zero Toffoli cost, whereas we do not. This is a straight factor of two difference. This leaves us with 144 vs 100. The extra factor of 44 arises from the different costs of the comparators listed in the paper and those in qualtran. The paper uses a cost of $n_m - 1$, whereas the qualtran comparators assume a cost of $n_m$ for comparison to a constant and $2 n_m - 1$ when comparing two registers.  
2. Rotations: The paper uses a phase gradient register which has a much lower cost than the qualtran cost which assumes a generic synthesis cost. 
3. Reflections and other: This discrepancy arises because the paper states the first reflection in between the comparators has Toffoli cost of 3, rather than what we have which is 2 Toffolis and 1 Multi-Controlled Z which costs 2 Toffolis. Our costs for the second reflection match. The other discrepancy is for the operations in between the second set of comparators. Here we count 2 Toffolis and 1 Multi-Controlled-Not gate of cost 2 Toffolis.  

The leading order Toffoli cost of this state preparation is 10 $n_m$ in the paper which arises from the comparators and the reflection on the $\mu$ and $\nu$ registers, i.e. $4(n_m - 1) + 4n_m - 3 + 2n_m -1 \approx 10 n_m$.

## `PrepareTHC`
State Preparation for THC Hamilontian.

Prepares the state

$$
    \frac{1}{\sqrt{\lambda}}|+\rangle|+\rangle\left[
        \sum_\ell^{N/2} \sqrt{t_\ell}|\ell\rangle|M+1\rangle
        + \frac{1}{\sqrt{2}} \sum_{\mu\le\nu}^M \sqrt{\zeta_{\mu\nu}} |\mu\rangle|\nu\rangle
    \right].
$$

Note we use UniformSuperpositionTHC as a subroutine as part of this bloq in
contrast to the reference which keeps them separate.

#### Parameters
 - `num_mu`: THC auxiliary index dimension $M$
 - `num_spin_orb`: number of spin orbitals $N$
 - `alt_mu`: Alternate values for mu indices.
 - `alt_nu`: Alternate values for nu indices.
 - `alt_theta`: Alternate values for theta indices.
 - `theta`: Signs of lcu coefficients.
 - `keep`: keep values.
 - `keep_bitsize`: number of bits for keep register for coherent alias sampling. 

Registers:
 - mu: $\mu$ register.
 - nu: $\nu$ register.
 - theta: sign register.
 - succ: success flag qubit from uniform state preparation
 - eq_nu_mp1: flag for if $nu = M+1$
 - plus_a / plus_b: plus state for controlled swaps on spins.

#### References
[Even more efficient quantum computations of chemistry through tensor hypercontraction](https://arxiv.org/pdf/2011.03494.pdf) Fig. 2 and Fig. 3.


In [ ]:
from qualtran.bloqs.chemistry.thc import PrepareTHC

num_spat = 20
num_mu = 20
t_l = np.random.randint(0, 10, size=num_spat)
zeta = np.random.randint(0, 10, size=(num_mu, num_mu))
zeta = 0.5 * (zeta + zeta.T)
eps = 1e-3
bloq = PrepareTHC.build(t_l, zeta, probability_epsilon=eps)
show_bloq(bloq.decompose_bloq())

In [ ]:
from qualtran.resource_counting import get_bloq_counts_graph, GraphvizCounts
from cirq_ft.algos.multi_control_multi_target_pauli import MultiControlPauli

graph, sigma = get_bloq_counts_graph(bloq, generalizer=generalize)
GraphvizCounts(graph).get_svg()

### ```Paper Comparison```

Let's compare our costs to those from the paper. Note we will only look at the cost of prepare. Inverting prepare has the same cost up to the cost of the inverse QROM being reduced to

$$
\lceil \frac{d}{k_{s2}} \rceil + k_{s2}
$$


In [ ]:
from qualtran.bloqs.chemistry.thc_notebook_utils import bin_bloq_counts

binned_counts = bin_bloq_counts(bloq)
data_size = bloq.num_mu * (bloq.num_mu + 1) // 2 + bloq.num_spin_orb // 2
num_bits_mu = bloq.signature[0].bitsize
qrom_bitsize = 2 * num_bits_mu + 2 + bloq.keep_bitsize
paper_costs = {
    'contiguous_register': 4*(num_bits_mu ** 2 + num_bits_mu - 1),
    'controlled_swaps': 4*(2 * num_bits_mu + (num_bits_mu + 1)), # Swaps from inequality and swap from from 
    'qrom': 4 * (int(np.ceil(data_size/4) + qrom_bitsize * (4 - 1))), # Eq. 31 from THC paper, k = 4 in this specific case.
    'comparator': 4*bloq.keep_bitsize,
}
for k, v in paper_costs.items():
    print(f"{k}: qualtran = {binned_counts[k]} vs paper cost = {v}.")

print(f"Total cost = {sum(v for v in binned_counts.values())}")

The main discrepancies arise from QROAM assumptions and the difference in comparator cost seen before. 